# Taller 2

Daniel Felipe Vargas Ulloa - 202123899
Andres Francisco Borda - 201729184
MINE-4101: Ciencia de Datos Aplicada

## 1. Preparación

In [1]:
# Importaciones necesarias para construir el modelo

# Manipulación de datos
import numpy as np
import pandas as pd

# Manejo de imágenes y visualización
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Librerías de machine learning y deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Configuración adicional
import os
import shutil
import warnings
warnings.filterwarnings('ignore')




Despues de importar las librerias necesarias pasamos a cargar los datos de entrenamiento procesados previamente en la etapa de entendimiento y preparación. Para realizar el análisis desde las capas especificas de productos y no desde las categorias mayores se re estructuran los datos en carpetas combinadas.

In [2]:
# Directorios originales y el directorio de destino
original_train_dir = 'GroceryStoreDataset/dataset/train_augmented/'
new_train_dir = 'GroceryStoreDataset/dataset/train_combined/'

# Crear el nuevo directorio de clases combinadas
if not os.path.exists(new_train_dir):
    os.makedirs(new_train_dir)

# Recorremos las carpetas principales (clases gruesas)
for coarse_class in os.listdir(original_train_dir):
    coarse_class_path = os.path.join(original_train_dir, coarse_class)
    
    # Verificamos si es una carpeta
    if os.path.isdir(coarse_class_path):
        
        # Recorremos las subcarpetas (clases finas)
        for fine_class in os.listdir(coarse_class_path):
            fine_class_path = os.path.join(coarse_class_path, fine_class)
            
            # Verificamos si es una carpeta
            if os.path.isdir(fine_class_path):
                
                # Creamos una nueva carpeta para la clase gruesa + fina
                new_class_name = f'{coarse_class}_{fine_class}'
                new_class_path = os.path.join(new_train_dir, new_class_name)
                
                if not os.path.exists(new_class_path):
                    os.makedirs(new_class_path)
                
                # Copiamos las imágenes de la clase fina a la nueva carpeta combinada
                for image_name in os.listdir(fine_class_path):
                    image_path = os.path.join(fine_class_path, image_name)
                    
                    # Verificamos si es una imagen
                    if os.path.isfile(image_path):
                        shutil.copy(image_path, new_class_path)

print("Estructura de directorios combinados creada con éxito.")

Estructura de directorios combinados creada con éxito.


In [3]:
# Directorios para los datos
train_dir = 'GroceryStoreDataset/dataset/train_augmented/'

# Definimos el generador de datos para cargar las imágenes de entrenamiento
train_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_directory(
    new_train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    shuffle=True
)

# Verificamos las clases combinadas
print(train_generator.class_indices)

Found 21120 images belonging to 81 classes.
{'Apple_Golden-Delicious': 0, 'Apple_Granny-Smith': 1, 'Apple_Pink-Lady': 2, 'Apple_Red-Delicious': 3, 'Apple_Royal-Gala': 4, 'Asparagus_Asparagus': 5, 'Aubergine_Aubergine': 6, 'Avocado_Avocado': 7, 'Banana_Banana': 8, 'Cabbage_Cabbage': 9, 'Carrots_Carrots': 10, 'Cucumber_Cucumber': 11, 'Garlic_Garlic': 12, 'Ginger_Ginger': 13, 'Juice_Bravo-Apple-Juice': 14, 'Juice_Bravo-Orange-Juice': 15, 'Juice_God-Morgon-Apple-Juice': 16, 'Juice_God-Morgon-Orange-Juice': 17, 'Juice_God-Morgon-Orange-Red-Grapefruit-Juice': 18, 'Juice_God-Morgon-Red-Grapefruit-Juice': 19, 'Juice_Tropicana-Apple-Juice': 20, 'Juice_Tropicana-Golden-Grapefruit': 21, 'Juice_Tropicana-Juice-Smooth': 22, 'Juice_Tropicana-Mandarin-Morning': 23, 'Kiwi_Kiwi': 24, 'Leek_Leek': 25, 'Lemon_Lemon': 26, 'Lime_Lime': 27, 'Mango_Mango': 28, 'Melon_Cantaloupe': 29, 'Melon_Galia-Melon': 30, 'Melon_Honeydew-Melon': 31, 'Melon_Watermelon': 32, 'Milk_Arla-Ecological-Medium-Fat-Milk': 33, 'Milk

In [4]:
# Directorios originales y el directorio de destino para val y test
original_valid_dir = 'GroceryStoreDataset/dataset/val/'
original_test_dir = 'GroceryStoreDataset/dataset/test/'

new_valid_dir = 'GroceryStoreDataset/dataset/val_combined/'
new_test_dir = 'GroceryStoreDataset/dataset/test_combined/'

# Función para reestructurar los datos
def restructure_data(original_dir, new_dir):
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

    # Recorremos las carpetas principales (Fruits, Vegetables, Packages)
    for category in os.listdir(original_dir):
        category_path = os.path.join(original_dir, category)

        # Verificamos si es una carpeta (Fruits, Vegetables, o Packages)
        if os.path.isdir(category_path):
            # Recorremos las subcarpetas (las clases gruesas como Apple, Banana, etc.)
            for coarse_class in os.listdir(category_path):
                coarse_class_path = os.path.join(category_path, coarse_class)

                # Verificamos si es una carpeta (como Apple, Banana, etc.)
                if os.path.isdir(coarse_class_path):
                    # Recorremos las subcarpetas (las clases finas como Golden-Delicious, etc.)
                    for fine_class in os.listdir(coarse_class_path):
                        fine_class_path = os.path.join(coarse_class_path, fine_class)

                        # Verificamos si es una carpeta (como Golden-Delicious, etc.)
                        if os.path.isdir(fine_class_path):
                            # Creamos la carpeta combinada con la clase gruesa y fina
                            new_class_name = f'{coarse_class}_{fine_class}'
                            new_class_path = os.path.join(new_dir, new_class_name)

                            if not os.path.exists(new_class_path):
                                os.makedirs(new_class_path)

                            # Copiamos las imágenes a la nueva carpeta combinada
                            for image_name in os.listdir(fine_class_path):
                                image_path = os.path.join(fine_class_path, image_name)
                                # Verificamos si es una imagen
                                if os.path.isfile(image_path):
                                    shutil.copy(image_path, new_class_path)

    print(f"Estructura de directorios combinados creada con éxito en {new_dir}.")

# Reestructuramos los directorios de validación y test
restructure_data(original_valid_dir, new_valid_dir)
restructure_data(original_test_dir, new_test_dir)

Estructura de directorios combinados creada con éxito en GroceryStoreDataset/dataset/val_combined/.
Estructura de directorios combinados creada con éxito en GroceryStoreDataset/dataset/test_combined/.


In [5]:
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_directory(
    new_valid_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    new_test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)

# Verificamos las clases combinadas
print(valid_generator.class_indices)
print(test_generator.class_indices)

Found 185 images belonging to 81 classes.


Found 1674 images belonging to 81 classes.
{'Apple_Golden-Delicious': 0, 'Apple_Granny-Smith': 1, 'Apple_Pink-Lady': 2, 'Apple_Red-Delicious': 3, 'Apple_Royal-Gala': 4, 'Asparagus_Asparagus': 5, 'Aubergine_Aubergine': 6, 'Avocado_Avocado': 7, 'Banana_Banana': 8, 'Cabbage_Cabbage': 9, 'Carrots_Carrots': 10, 'Cucumber_Cucumber': 11, 'Garlic_Garlic': 12, 'Ginger_Ginger': 13, 'Juice_Bravo-Apple-Juice': 14, 'Juice_Bravo-Orange-Juice': 15, 'Juice_God-Morgon-Apple-Juice': 16, 'Juice_God-Morgon-Orange-Juice': 17, 'Juice_God-Morgon-Orange-Red-Grapefruit-Juice': 18, 'Juice_God-Morgon-Red-Grapefruit-Juice': 19, 'Juice_Tropicana-Apple-Juice': 20, 'Juice_Tropicana-Golden-Grapefruit': 21, 'Juice_Tropicana-Juice-Smooth': 22, 'Juice_Tropicana-Mandarin-Morning': 23, 'Kiwi_Kiwi': 24, 'Leek_Leek': 25, 'Lemon_Lemon': 26, 'Lime_Lime': 27, 'Mango_Mango': 28, 'Melon_Cantaloupe': 29, 'Melon_Galia-Melon': 30, 'Melon_Honeydew-Melon': 31, 'Melon_Watermelon': 32, 'Milk_Arla-Ecological-Medium-Fat-Milk': 33, 'Milk_

Basado en lo encontrado, podemos ver que el set de entrenamiento tiene fotos de productos que los otros sets no. Para poder corre el modelo efetivamente sera necesario añadir asi sea vacias estas clases a los otros sets asi que se realiza este ejercicio de emparejamiento.

In [6]:
# Directorios de train, val y test
train_dir = 'GroceryStoreDataset/dataset/train_combined/'
valid_dir = 'GroceryStoreDataset/dataset/val_combined/'
test_dir = 'GroceryStoreDataset/dataset/test_combined/'

# Obtener las clases del conjunto de entrenamiento
train_classes = set(os.listdir(train_dir))

# Función para crear clases vacías y eliminar clases no deseadas
def adjust_classes(original_dir, train_classes):
    # Recorremos las carpetas del directorio original (val o test)
    for category in os.listdir(original_dir):
        category_path = os.path.join(original_dir, category)
        
        if os.path.isdir(category_path):
            # Si la clase no está en el conjunto de entrenamiento, la eliminamos
            if category not in train_classes:
                shutil.rmtree(category_path) 
            else:
                # Si la clase no tiene imágenes, la creamos como vacía
                if len(os.listdir(category_path)) == 0:
                    print(f"Creando clase vacía: {category}")

    # Ahora, nos aseguramos de que todas las clases de entrenamiento estén presentes como carpetas en val y test
    for train_class in train_classes:
        class_path = os.path.join(original_dir, train_class)
        if not os.path.exists(class_path):
            print(f"Creando clase vacía en {original_dir}: {train_class}")
            os.makedirs(class_path)

# Ajustar las clases en validación y prueba
adjust_classes(valid_dir, train_classes)
adjust_classes(test_dir, train_classes)

print("Ajuste de clases completado.")

Creando clase vacía: Apple_Red-Delicious
Creando clase vacía: Asparagus_Asparagus
Creando clase vacía: Aubergine_Aubergine
Creando clase vacía: Avocado_Avocado
Creando clase vacía: Banana_Banana
Creando clase vacía: Cabbage_Cabbage
Creando clase vacía: Carrots_Carrots
Creando clase vacía: Cucumber_Cucumber
Creando clase vacía: Garlic_Garlic
Creando clase vacía: Ginger_Ginger
Creando clase vacía: Juice_Tropicana-Apple-Juice
Creando clase vacía: Juice_Tropicana-Golden-Grapefruit
Creando clase vacía: Juice_Tropicana-Juice-Smooth
Creando clase vacía: Juice_Tropicana-Mandarin-Morning
Creando clase vacía: Kiwi_Kiwi
Creando clase vacía: Leek_Leek
Creando clase vacía: Lemon_Lemon
Creando clase vacía: Lime_Lime
Creando clase vacía: Mango_Mango
Creando clase vacía: Milk_Arla-Lactose-Medium-Fat-Milk
Creando clase vacía: Mushroom_Brown-Cap-Mushroom
Creando clase vacía: Nectarine_Nectarine
Creando clase vacía: Orange_Orange
Creando clase vacía: Papaya_Papaya
Creando clase vacía: Passion-Fruit_Passi

In [7]:
# Directorios ajustados
train_dir = 'GroceryStoreDataset/dataset/train_combined/'
valid_dir = 'GroceryStoreDataset/dataset/val_combined/'
test_dir = 'GroceryStoreDataset/dataset/test_combined/'

# Creación de los generadores para entrenamiento, validación y prueba
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Generador de entrenamiento
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128), 
    batch_size=32,
    class_mode='sparse', 
    shuffle=True  
)

# Generador de validación
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,  
    target_size=(128, 128), 
    batch_size=32,  
    class_mode='sparse', 
    shuffle=True 
)

# Generador de prueba
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128), 
    batch_size=32,  
    class_mode='sparse',  
    shuffle=False 
)

# Verificamos las clases cargadas
print(f"Clases de entrenamiento: {train_generator.class_indices}")
print(f"Clases de validación: {valid_generator.class_indices}")
print(f"Clases de prueba: {test_generator.class_indices}")

Found 21120 images belonging to 81 classes.
Found 185 images belonging to 81 classes.
Found 1674 images belonging to 81 classes.
Clases de entrenamiento: {'Apple_Golden-Delicious': 0, 'Apple_Granny-Smith': 1, 'Apple_Pink-Lady': 2, 'Apple_Red-Delicious': 3, 'Apple_Royal-Gala': 4, 'Asparagus_Asparagus': 5, 'Aubergine_Aubergine': 6, 'Avocado_Avocado': 7, 'Banana_Banana': 8, 'Cabbage_Cabbage': 9, 'Carrots_Carrots': 10, 'Cucumber_Cucumber': 11, 'Garlic_Garlic': 12, 'Ginger_Ginger': 13, 'Juice_Bravo-Apple-Juice': 14, 'Juice_Bravo-Orange-Juice': 15, 'Juice_God-Morgon-Apple-Juice': 16, 'Juice_God-Morgon-Orange-Juice': 17, 'Juice_God-Morgon-Orange-Red-Grapefruit-Juice': 18, 'Juice_God-Morgon-Red-Grapefruit-Juice': 19, 'Juice_Tropicana-Apple-Juice': 20, 'Juice_Tropicana-Golden-Grapefruit': 21, 'Juice_Tropicana-Juice-Smooth': 22, 'Juice_Tropicana-Mandarin-Morning': 23, 'Kiwi_Kiwi': 24, 'Leek_Leek': 25, 'Lemon_Lemon': 26, 'Lime_Lime': 27, 'Mango_Mango': 28, 'Melon_Cantaloupe': 29, 'Melon_Galia-Mel

## 2. Construcción del modelo

In [8]:
model = models.Sequential([
    # Capa 1
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02), input_shape=(128, 128, 3)),
    BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Capa 2
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.02)),
    BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # Capa 3
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.02)),
    BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # Aplanar y capas densas
    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_regularizer=l2(0.02)),
    layers.Dropout(0.6),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Compilación con tasa de aprendizaje más baja
model.compile(optimizer=Adam(learning_rate=0.00005), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Entrenamiento
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=valid_generator,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluar en el conjunto de prueba
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test accuracy: {(test_acc):.4f}")

Epoch 1/30


660/660 [==============================] - 178s 268ms/step - loss: 14.9286 - accuracy: 0.1340 - val_loss: 13.2532 - val_accuracy: 0.1081
Epoch 2/30
660/660 [==============================] - 176s 267ms/step - loss: 10.7656 - accuracy: 0.3619 - val_loss: 10.3932 - val_accuracy: 0.2108
Epoch 3/30
660/660 [==============================] - 175s 265ms/step - loss: 8.1191 - accuracy: 0.5485 - val_loss: 8.7531 - val_accuracy: 0.2919
Epoch 4/30
660/660 [==============================] - 173s 262ms/step - loss: 6.4136 - accuracy: 0.6699 - val_loss: 7.7641 - val_accuracy: 0.2973
Epoch 5/30
660/660 [==============================] - 173s 263ms/step - loss: 5.2023 - accuracy: 0.7555 - val_loss: 6.7698 - val_accuracy: 0.3297
Epoch 6/30
660/660 [==============================] - 176s 266ms/step - loss: 4.3167 - accuracy: 0.8143 - val_loss: 6.1959 - val_accuracy: 0.3405
Epoch 7/30
660/660 [==============================] - 166s 252ms/step - loss: 3.6301 - accuracy: 0.8611 - val_loss: 5.

## 3. Análisis de Resultados del modelo

El modelo CNN entrenado para la clasificación de productos muestra un rendimiento excepcional en el conjunto de entrenamiento, alcanzando un 97% de precisión. Esto sugiere que el modelo ha logrado aprender de manera muy efectiva las características presentes en los datos de entrenamiento, capturando patrones complejos y distinguiendo entre diferentes clases de productos.

Sin embargo, al evaluar el modelo en el conjunto de validación, se observa una disminución significativa en su rendimiento, lo que indica un problema de sobreajuste. Aunque el modelo alcanza un 80% de precisión en el conjunto de prueba, un resultado notable, la disparidad entre los resultados de entrenamiento y validación sugiere que el modelo está demasiado adaptado a los datos de entrenamiento y no generaliza completamente a datos nuevos.

El sobreajuste puede deberse a varios factores, como una arquitectura compleja del modelo en relación con el tamaño del conjunto de datos o una falta de regularización adecuada. Esto implica que, aunque el modelo es robusto en condiciones controladas, puede no ser completamente fiable frente a datos del mundo real o productos que no estén representados en el conjunto de entrenamiento.

### Metrica de desempeño Accuracy

In [ ]:
print(f"Test accuracy: {(test_acc):.4f}")

Test accuracy: 0.8243


El 80% de precisión obtenido por el modelo CNN en el conjunto de prueba refleja su capacidad para clasificar correctamente 8 de cada 10 productos en el contexto de uso real. Este nivel de precisión es notablemente alto para una tarea compleja como la clasificación de productos a nivel de marca, donde muchas categorías son visualmente similares. Este resultado se utiliza como punto de comparación con el otro modelo evaluado.

La precisión es la métrica más adecuada para este proyecto, ya que mide directamente el porcentaje de predicciones correctas sobre el total de predicciones realizadas. Esto es crucial en el contexto del supermercado, donde cada predicción incorrecta puede impactar negativamente la experiencia del cliente, al requerir asistencia manual y aumentar el tiempo de atención. Un modelo con alta precisión asegura que el sistema automático sea confiable, reduciendo errores y aumentando la eficiencia operativa en el proceso de checkout.